In [1]:
import torch

In [2]:
torch.cuda.is_available()

True

In [6]:
torch.cuda.get_device_name(torch.cuda.current_device())

'GeForce GTX 1050'

In [10]:
print(f"{torch.cuda.memory_allocated()} bytes allocated")

0 bytes allocated


In [9]:
torch.cuda.memory_reserved()

0

In [11]:
a = torch.FloatTensor([1.0,2.0])

In [14]:
a.device # we want thos to be in gpu

device(type='cpu')

In [15]:
b = torch.FloatTensor([1.0,2.0]).cuda()

In [16]:
b.device

device(type='cuda', index=0)

In [18]:
print(f"{torch.cuda.memory_allocated()} bytes of memory is not being used to store the tensor b")

512 bytes of memory is not being used to store the tensor b


In [19]:
from PIL import Image
import os
from IPython.display import display

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
%matplotlib inline

In [20]:
train_transform = transforms.Compose([transforms.RandomRotation(degrees=10),
                                       transforms.RandomHorizontalFlip(p = 0.5),
                                       transforms.Resize((224,224)),
                                      transforms.CenterCrop((224,224)),
                                      transforms.ToTensor(), 
                                      transforms.Normalize(mean = [0.485,0.456,0.406], std = [0.229,0.224,0.225])])

test_transform = transforms.Compose([transforms.Resize((224,224)),
                                      transforms.CenterCrop((224,224)),
                                      transforms.ToTensor(), 
                                      transforms.Normalize(mean = [0.485,0.456,0.406], std = [0.229,0.224,0.225])])



In [27]:
root = r"D:\PyTorch for Deep Learning with Python Bootcamp\1. Course Overview, Installs, and Setup\PYTORCH_NOTEBOOKS\Images\CATS_DOGS"

torch.manual_seed(1429)
train_data = datasets.ImageFolder(os.path.join(root, 'train'), transform = train_transform)
test_data = datasets.ImageFolder(os.path.join(root, 'test'), transform = test_transform)
train_loader = DataLoader(train_data,batch_size = 10, shuffle = True,pin_memory = True )
test_loader = DataLoader(test_data,batch_size = 10, shuffle = False,pin_memory = True)
class_names = train_data.classes

# for img, lbl in train_loader:
#     break

# img.shape

# inv_normalize = transforms.Compose([transforms.Normalize(mean= [-0.485/0.229,-0.456/0.224,-0.406/0.225], std = [1/0.229,1/0.224,1/0.225])])
# im = inv_normalize(img[5])
# plt.imshow(np.transpose((im.numpy()), (1,2,0)))

# Model

# class CNN(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=3, stride=1) #out_channel = no of filters, kernel_size = filter dim 
#         self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=3, stride=1)
#         self.fc1 = nn.Linear(54*54*16,120) # out_channels from conv1, in/out_channel from conv2
#         self.fc2 = nn.Linear(120,84)
#         self.fc3 = nn.Linear(84,2) # output as 10 classes
        
#     def forward(self,X):
#         X = F.relu(self.conv1(X))
#         X = F.max_pool2d(X,2,2)
#         X = F.relu(self.conv2(X))
#         X = F.max_pool2d(X,2,2)
#         X = X.view(-1,54*54*16)
#         X = F.relu(self.fc1(X))
#         X = F.relu(self.fc2(X))
#         X = self.fc3(X)
#         return F.log_softmax(X, dim = 1)
alex_net = models.alexnet(pretrained=True).cuda()
for param in alex_net.parameters():
    param.requires_grad = False
torch.manual_seed(1429)
alex_net.classifier = nn.Sequential(nn.Linear(in_features=9216, out_features=1024),
                                   nn.ReLU(),
                                   nn.Dropout(p = 0.5),
                                   nn.Linear(in_features=1024, out_features=2),
                                   nn.LogSoftmax(dim = 1)) # nn.Sequential allows us to quickly add layers
alex_net = models.alexnet(pretrained=True).cuda()


torch.manual_seed(1429)
# model = alex_net().cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(alex_net.classifier.parameters(), lr = 1e-3)


# for param in model.parameters():

#     print(param.numel())


epochs = 1
train_loss = []
test_loss = []
train_correct = []
test_correct = []

for i in range(epochs):
    trn_corr = 0
    test_corr = 0
    
    
    for batch, (X_train, y_train) in enumerate(train_loader):
        X_train = torch.FloatTensor(X_train).cuda()
        y_train = torch.LongTensor(y_train).cuda()
        batch = batch+1
        y_pred = alex_net(X_train)
        losses = criterion(y_pred,y_train)
        pred = torch.max(y_pred.data,1)[1]
        
        batch_corr = (pred == y_train).sum() # True 1 / False 0
        
        trn_corr = trn_corr+batch_corr
         
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()
        
        if batch%600 == 0:
            print(f'Epoch {i} batch {batch} and loss {losses} and accuracy is {trn_corr.item()*100/(10*batch)}%')
    train_loss.append(losses)
    train_correct.append(trn_corr)
    
    with torch.no_grad():
        for batch, (X_test, y_test) in enumerate(test_loader):
            X_test = torch.FloatTensor(X_test).cuda()
            y_test = torch.LongTensor(y_test).cuda()
            y_val = model(X_test)
            
            predicted = torch.max(y_val.data,1)[1]
            
            test_corr = test_corr + (predicted == y_test).sum()
        losses = criterion(y_val, y_test)
        test_loss.append(losses)
        test_correct.append(test_corr)

Epoch 0 batch 600 and loss 1.0463640689849854 and accuracy is 88.48333333333333%
Epoch 0 batch 1200 and loss 0.00056585727725178 and accuracy is 89.7%
Epoch 0 batch 1800 and loss 0.006783259566873312 and accuracy is 90.68888888888888%
